In [31]:
#Import necessary modules
import pandas as pd
import PySimpleGUI as sg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
import plotly.figure_factory as ff
import plotly.io as pio

In [32]:
#Plots show up externally and are interactive
%matplotlib qt

In [33]:
#Popup window to select .csv file for analysis
sg.theme('LightBlue2')

File = sg.popup_get_file('Please select .csv file for analyzing',
                          title = 'Select File',
                          keep_on_top = True)
Data = pd.read_csv(File,
                    header = 0, 
                    keep_default_na = False)
#Keep na so athlete NA does not become NaN

In [34]:
#Fill blanks with NaN's
Data.replace(r'^\s*$', np.nan, regex = True, inplace = True)

#Create New df, remove NaN, and sort
Data = Data[['Date', 'ID', 'A:C Ratio']].copy()
Data = Data.dropna()
Data.sort_values('ID', inplace = True, ascending = True)
Data['Date'] = pd.to_datetime(Data['Date'])
Data['A:C Ratio'] = pd.to_numeric(Data['A:C Ratio'])

In [35]:
#Creating a dictionary of athletes
dict_of_athletes = {k: v for k, v in Data.groupby('ID')}

In [36]:
#Visualizing athlete's A:C Ratios over time

for key, value in dict_of_athletes.items():
    fig = plt.figure(figsize = (8,5))
    #Set figure size to 8X5 and dpi to 300
    y = dict_of_athletes[key]['A:C Ratio']
    x = dict_of_athletes[key]['Date']
    colors = np.where(dict_of_athletes[key]['A:C Ratio'] >= 2.5, 'r', 'b')
    #Highlight all ratios 2.5 and above as red
    plt.scatter(x,y, c = colors)
    locator = mdate.MonthLocator()
    plt.gca().xaxis.set_major_locator(locator)
    plt.xticks(rotation = 45)
    plt.xlabel('Date')
    plt.ylabel('A:C Ratio')
    plt.title(key)
    plt.gca().spines['right'].set_color('none')
    plt.gca().spines['top'].set_color('none')
plt.show()

In [37]:
#Table with A:C Ratio above 2.5
Criteria = Data['A:C Ratio'] >= 2.5
NewData = Data[Criteria]

#Opens table in Browser
pio.renderers.default = 'browser'

fig = ff.create_table(NewData)
fig.show()

In [38]:
#Set index to before saving to excel
NewData.set_index('Date', inplace = True)

#Save table in excel worksheet
selectfolder = sg.popup_get_folder('Select a folder to save all plots and files', keep_on_top = True)
with pd.ExcelWriter(selectfolder + '/Analyzed Wellness Data.xlsx') as writer:
    NewData.to_excel(writer, sheet_name = 'AC Ratio Data')